# santa 2019 : Test Gurobi from sample

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB

import pickle

np.random.seed(42)

In [3]:
PATH_INPUT = 'kaggle/input'
#submission_test = pd.read_csv('submission_72060.8_sps.csv', 
#                         index_col='family_id')
fpath = PATH_INPUT + '/santa-2019-workshop-scheduling/sample_submission.csv'
submission_test = pd.read_csv(fpath, index_col='family_id')
arr_curr = submission_test["assigned_day"].values

In [4]:
arr_curr

array([100,  99,  98, ...,   3,   2,   1], dtype=int64)

In [ ]:


# Our initial (current best) solution
initial = arr_curr

# Read in some data
data = pd.read_csv('family_data.csv', index_col=0)
NUM_FAMILY = len(data)
NUM_CHOICES = 5
NUM_DAY = 100
MIN_OCCUPANCY, MAX_OCCUPANCY = 125, 300
NUM_OCCUPANCY = MAX_OCCUPANCY - MIN_OCCUPANCY + 1
COUNT = np.arange(MIN_OCCUPANCY, MAX_OCCUPANCY + 1)

family_size_dict = data[['n_people']].to_dict()['n_people']
cols = [f'choice_{i}' for i in range(NUM_CHOICES)]
choices = data[cols].values

# Initialize a 5000x5 matrix with the preference costs for the families
# PREF_COSTS[f][i] contains the preference cost if the f-th family
# is assigned their i-th choice
PREF_COSTS = np.zeros((5000, 5))
for f in range(0, 5000):
    n = family_size_dict[f]
    PREF_COSTS[f] = [
        0,
        50,
        50 + 9 * n,
        100 + 9 * n,
        200 + 9 * n
    ]

# Initialize a 176x176 matrix for the accounting costs.
# ACC_COST[k][l] contains the accounting cost if the occupancy
# of day i == k and that of day i + 1 == l
ACC_COSTS = np.zeros((176, 176))
for i in range(125, 301):
    for j in range(125, 301):
        constant = (i - 125) / 400
        diff = abs(i - j)
        ACC_COSTS[i - 125, j - 125] = constant * i ** (.5 + diff / 50)

def print_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        solution = []
        obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
        for f in range(NUM_FAMILY):
            for c in range(NUM_CHOICES):
                if model.cbGetSolution(model.getVarByName('fam_{}_choice_{}'.format(f, c))):
                    solution.append(choices[f][c])
                    break
        print(solution)
        pickle.dump(solution, open('mip_{}.p'.format(int(obj)), 'wb+'))

# Create a new model
m = gp.Model("santa-workshop")

# Create a 5000x5 matrix with the assignments.
# asm[f][i] == 1 if family f is assigned choice i
asm = []
for f in range(NUM_FAMILY):
    asm_i = []
    for c in range(NUM_CHOICES):
        asm_i.append(m.addVar(vtype=GRB.BINARY, name='fam_{}_choice_{}'.format(f, c)))
    asm.append(asm_i)

# Create a 100x176x176 matrix with the occupancies.
# occ[d][npp1][npp2] == 1 if (npp1 + 125) people go to (day + 1) and (npp2 + 125) people go to (day + 2)
occ = []
for d in range(NUM_DAY):
    occ_d = []
    for npp1 in range(NUM_OCCUPANCY):
        occ_d_npp1 = []
        for npp2 in range(NUM_OCCUPANCY):
            occ_d_npp1.append(m.addVar(vtype=GRB.BINARY, name='day_{}_occ_{}_{}'.format(d + 1, npp1 + MIN_OCCUPANCY, npp2 + MIN_OCCUPANCY)))
        occ_d.append(occ_d_npp1)
    occ.append(occ_d)

# Store the occupancies in a dict
occupancy = {}
for d in range(NUM_DAY):
    occupancy[d] = gp.quicksum(asm[f][c] * family_size_dict[f]
                               for c in range(NUM_CHOICES) for f in range(NUM_FAMILY)
                               if choices[f][c] == (d + 1))
occupancy[NUM_DAY] = occupancy[NUM_DAY - 1]

# Each family should get assigned exactly one choice
for f in range(NUM_FAMILY):
    m.addConstr(gp.quicksum(asm[f][c] for c in range(NUM_CHOICES)) == 1, name='1_choice_{}'.format(f))

for d in range(NUM_DAY):
    # Occupancies should be between 125 and 300 per day
    m.addConstr(occupancy[d] >= MIN_OCCUPANCY, name='min_occ_{}'.format(d))
    m.addConstr(occupancy[d] <= MAX_OCCUPANCY, name='max_occ_{}'.format(d))

    # Create indicator (binary) variables corresponding to the value
    y_sum_npp1 = gp.quicksum(occ[d][npp1][npp2] * COUNT[npp1]
                             for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
    y_sum_npp2 = gp.quicksum(occ[d][npp1][npp2] * COUNT[npp2]
                             for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
    
    m.addConstr(y_sum_npp1 == occupancy[d], name='indicator_npp1_{}'.format(d))
    m.addConstr(y_sum_npp2 == occupancy[d + 1], name='indicator_npp2_{}'.format(d))

    # Exactly 1 indicator variable should be set
    y_sum = gp.quicksum(occ[d][npp1][npp2] for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
    m.addConstr(y_sum == 1, name='1_indicator_{}'.format(d))

# Make sure the binary indicators are consistent
for d in range(NUM_DAY - 1):
    for t in range(NUM_OCCUPANCY):
        y_sum_npp1 = gp.quicksum(occ[d][npp1][t] for npp1 in range(NUM_OCCUPANCY))
        y_sum_npp2 = gp.quicksum(occ[d + 1][t][npp2] for npp2 in range(NUM_OCCUPANCY))
        m.addConstr(y_sum_npp1 == y_sum_npp2, name='consistent_occ_{}_{}'.format(d, t))


# Initialize solution
init_occs = np.zeros(NUM_DAY + 1, dtype=int)
for f in range(NUM_FAMILY):
    init_occs[initial[f] - 1] += family_size_dict[f]
init_occs[-1] = init_occs[-2]

for f in range(NUM_FAMILY):
    for c in range(NUM_CHOICES):
        if choices[f][c] == initial[f]:
            asm[f][c].start = 1
        else:
            asm[f][c].start = 0

for d in range(NUM_DAY):
    for npp1 in range(NUM_OCCUPANCY):
        for npp2 in range(NUM_OCCUPANCY):
            if init_occs[d] == npp1 + MIN_OCCUPANCY and init_occs[d + 1] == npp2 + MIN_OCCUPANCY:
                occ[d][npp1][npp2].start = 1
            else:
                occ[d][npp1][npp2].start = 0

# Set objective
pref_cost = gp.quicksum(asm[f][c]*PREF_COSTS[f][c] for c in range(NUM_CHOICES) for f in range(NUM_FAMILY))
acc_cost = gp.quicksum(occ[d][npp1][npp2]*ACC_COSTS[npp1][npp2]
                       for d in range(NUM_DAY) for npp1 in range(NUM_OCCUPANCY) for npp2 in range(NUM_OCCUPANCY))
m.setObjective(pref_cost + acc_cost, GRB.MINIMIZE)

# Optimize model
m.optimize(print_callback)

m.setParam('MIPGap', 0)

input()

print(m.objVal)
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))


Using license file c:\gurobi901\gurobi.lic
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 22924 rows, 3122600 columns and 15549486 nonzeros
Model fingerprint: 0x0e95bbfd
Variable types: 0 continuous, 3122600 integer (3122600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-02, 4e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

User MIP start did not produce a new incumbent solution
User MIP start violates constraint 1_choice_0 by 1.000000000

Presolve removed 702 rows and 223044 columns (presolve time = 5s) ...
Presolve removed 702 rows and 233546 columns (presolve time = 10s) ...
Presolve removed 702 rows and 233590 columns (presolve time = 15s) ...
Presolve removed 702 rows and 233590 columns (presolve time = 20s) ...
Presolve removed 802 rows and 233590 columns (presolve time = 27s) 

  82   6.73516129e+04  6.72663669e+04  4.59e-09 6.67e-05  1.48e-05   227s
  83   6.73325244e+04  6.72704900e+04  3.33e-09 5.41e-05  1.08e-05   228s
  84   6.73230541e+04  6.72914253e+04  1.83e-08 5.26e-05  5.48e-06   229s
  85   6.73138227e+04  6.73078001e+04  7.35e-08 6.36e-05  1.04e-06   230s
  86   6.73102988e+04  6.73089888e+04  8.66e-07 3.68e-05  2.27e-07   230s
  87   6.73092142e+04  6.73091078e+04  1.10e-07 3.82e-05  1.84e-08   231s
  88   6.73092114e+04  6.73091080e+04  7.07e-07 4.65e-05  1.78e-08   232s
  89   6.73091145e+04  6.73091141e+04  1.18e-08 4.79e-05  8.80e-11   233s

Barrier solved model in 89 iterations and 233.32 seconds
Optimal objective 6.73091145e+04


Root crossover log...

   21513 DPushes remaining with DInf 6.1571838e-03               234s
     259 DPushes remaining with DInf 0.0000000e+00               235s
       0 DPushes remaining with DInf 0.0000000e+00               236s

      94 Dual superbasic variables remain

      36 PPushes remaining with PInf 2

H    0     0                    3158473.1349 67309.1140  97.9%     -  259s
[52, 26, 100, 2, 53, 32, 88, 25, 18, 88, 92, 19, 98, 54, 45, 22, 46, 47, 75, 3, 3, 56, 61, 19, 75, 16, 58, 38, 45, 8, 68, 74, 24, 32, 46, 31, 47, 15, 13, 10, 25, 92, 81, 11, 49, 5, 15, 45, 32, 33, 50, 28, 81, 37, 69, 91, 12, 31, 74, 39, 98, 18, 39, 63, 3, 47, 66, 94, 85, 5, 32, 71, 66, 10, 47, 53, 79, 20, 52, 37, 18, 17, 32, 88, 9, 54, 29, 25, 57, 23, 54, 17, 85, 5, 47, 24, 75, 96, 3, 39, 39, 53, 29, 68, 47, 12, 34, 41, 34, 17, 22, 68, 13, 18, 15, 22, 77, 94, 69, 4, 56, 17, 24, 95, 14, 47, 3, 2, 56, 43, 16, 11, 76, 54, 18, 5, 88, 82, 2, 89, 61, 89, 26, 38, 28, 18, 64, 28, 63, 61, 60, 3, 88, 3, 25, 100, 95, 87, 13, 55, 40, 47, 86, 84, 18, 97, 84, 93, 19, 67, 12, 24, 21, 55, 34, 17, 52, 74, 80, 2, 20, 7, 94, 17, 4, 80, 50, 43, 10, 51, 9, 54, 77, 35, 95, 55, 100, 59, 90, 51, 26, 35, 23, 38, 67, 1, 28, 47, 1, 71, 23, 67, 10, 52, 31, 1, 5, 5, 38, 68, 1, 17, 36, 60, 4, 61, 59, 49, 87, 48, 57, 48, 47, 87, 82, 53, 37, 3

     0     0 67472.8388    0  482 3155771.03 67472.8388  97.9%     -  772s
     0     0 67531.8688    0  541 3155771.03 67531.8688  97.9%     -  915s
     0     0 67541.6598    0  547 3155771.03 67541.6598  97.9%     - 1019s
     0     0 67541.8036    0  550 3155771.03 67541.8036  97.9%     - 1033s
     0     0 67631.9415    0  531 3155771.03 67631.9415  97.9%     - 1183s
     0     0 67631.9663    0  532 3155771.03 67631.9663  97.9%     - 1221s
     0     0 67699.5920    0  509 3155771.03 67699.5920  97.9%     - 1271s
     0     0 67699.7450    0  522 3155771.03 67699.7450  97.9%     - 1284s
     0     0 67699.7560    0  517 3155771.03 67699.7560  97.9%     - 1293s
     0     0 67759.0827    0  567 3155771.03 67759.0827  97.9%     - 1502s
     0     0 67879.8740    0  603 3155771.03 67879.8740  97.8%     - 1831s
     0     0 67886.2528    0  632 3155771.03 67886.2528  97.8%     - 1903s
     0     0 67887.5213    0  648 3155771.03 67887.5213  97.8%     - 1948s
     0     0 67887.7980  

H    0     0                    3136385.9544 67945.2028  97.8%     - 4509s
     0     0 67978.6863    0  694 3136385.95 67978.6863  97.8%     - 4618s
     0     0 67984.8210    0  718 3136385.95 67984.8210  97.8%     - 4686s
     0     0 67985.2840    0  736 3136385.95 67985.2840  97.8%     - 4710s
     0     0 67985.3392    0  751 3136385.95 67985.3392  97.8%     - 4727s
     0     0 67997.1451    0  779 3136385.95 67997.1451  97.8%     - 4885s
     0     0 68004.7088    0  829 3136385.95 68004.7088  97.8%     - 5463s
     0     0 68005.7192    0  826 3136385.95 68005.7192  97.8%     - 5495s
     0     0 68005.9898    0  831 3136385.95 68005.9898  97.8%     - 5525s
     0     0 68006.0347    0  848 3136385.95 68006.0347  97.8%     - 5536s
     0     0 68012.3301    0  801 3136385.95 68012.3301  97.8%     - 5686s
     0     0 68018.8354    0  816 3136385.95 68018.8354  97.8%     - 5794s
     0     0 68019.5316    0  841 3136385.95 68019.5316  97.8%     - 5870s
     0     0 68019.6262  

...

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "<ipython-input-5-22c595622d4c>", line 42, in print_callback
KeyboardInterrupt


KeyboardInterrupt: 

Exception ignored in: 'gurobipy.callbackstub'
Traceback (most recent call last):
  File "callback.pxi", line 180, in gurobipy.CallbackClass.callback
  File "<ipython-input-5-22c595622d4c>", line 42, in print_callback
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
"Nodes       |    Current Node    |     Objective Bounds      |     Work"
"Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time"